In [70]:
import os
import pandas as pd
import re
import numpy as np

#data from http://times.cs.uiuc.edu/~wang296/Data/
#print(os.listdir('./Review_Texts'))

In [31]:
content = ''
with open('./Review_Texts/hotel_72572.dat', 'r') as content_file:
    content = content_file.read()
    
#print(content)
reviews = content.split("\n\n")
hotel = reviews[0]
reviews = reviews[1:]
print(hotel)
print()
firstReview = reviews[1]
print(firstReview)

<Overall Rating>4
<Avg. Price>$173
<URL>http://www.tripadvisor.com/ShowUserReviews-g60878-d72572-r23327047-Best_Western_Pioneer_Square_Hotel-Seattle_Washington.html

<Author>RW53
<Content>Location! Location?       view from room of nearby freeway 
<Date>Dec 26, 2008
<No. Reader>-1
<No. Helpful>-1
<Overall>3
<Value>4
<Rooms>3
<Location>2
<Cleanliness>4
<Check in / front desk>3
<Service>-1
<Business service>-1


In [50]:
#what categories do we have

cats = set()

for r in reviews:
    #selector= r'<([A-Z][a-z]+)>'
    selector= r'<(.*)>'
    matches = re.findall(selector, r)
    #print(matches)
    cats.update(matches)


cats = list(cats)
for c in cats:
    if "img src=" in c:
        cats.remove(c)
print(cats)

['Date', 'No. Reader', 'No. Helpful', 'Cleanliness', 'Content', 'Check in / front desk', 'Author', 'Value', 'Overall', 'Service', 'Business service', 'Rooms', 'Location']


In [83]:
#put data in pandas DF
intColumns = ['No. Reader', 'No. Helpful', 'Cleanliness','Check in / front desk', 'Value', 'Overall', 'Service', 'Business service', 'Rooms', 'Location']


data = pd.DataFrame(columns=cats)
for r in reviews:
    #print(r)
    thisReview = pd.Series([None]*len(cats), cats)
    splt = r.split("\n")
    for s in splt:
        for c in cats:
            if "<"+c+">" in s:
                value = s.replace('<'+c+'>', '')
                if c in intColumns:
                    value = int(value)
                if value == -1: #we dont want -1 as this is going to mess up averaging, take np.nan
                    value = np.nan
                
                
                thisReview[c] = value
    #print(thisReview)
    data = data.append(thisReview, ignore_index=True)

In [84]:
data

,Date,No. Reader,No. Helpful,Cleanliness,Content,Check in / front desk,Author,Value,Overall,Service,Business service,Rooms,Location
0,"Jan 6, 2009",NaN,NaN,5.0,Old seattle getaway This was Old World Excelle...,5.0,everywhereman2,5.0,5.0,5.0,5.0,5.0,5.0
1,"Dec 26, 2008",NaN,NaN,4.0,Location! Location? view from room of ne...,3.0,RW53,4.0,3.0,NaN,NaN,3.0,2.0
2,"Dec 14, 2008",NaN,NaN,5.0,"Wow, what charm! As a Travel Agent, I've staye...",4.0,KGBT,4.0,5.0,NaN,4.0,5.0,5.0
3,"Dec 1, 2008",NaN,NaN,5.0,Great location for sporting events We attend S...,5.0,Marilyn1949,4.0,4.0,5.0,5.0,4.0,5.0
4,"Nov 28, 2008",NaN,NaN,5.0,Pioneer Square BW Fine Choice We chose BW Pion...,5.0,MissViolet105,5.0,4.0,5.0,2.0,5.0,5.0
5,"Nov 25, 2008",NaN,NaN,5.0,Quality hotel at great price Very clean. Free ...,5.0,fallriverma,5.0,5.0,5.0,5.0,5.0,5.0
6,"Nov 18, 2008",NaN,NaN,4.0,"comfortable, good value, convenient location T...",4.0,simmotours,4.0,4.0,4.0,NaN,3.0,4.0
7,"Nov 15, 2008",NaN,NaN,NaN,Warm hospitality on a rainy Seattle night We h...,NaN,SweetwaterMill,NaN,5.0,NaN,NaN,NaN,NaN
8,"Nov 13, 2008",NaN,NaN,5.0,Pleasant stay This is an old hotel in the orig...,5.0,AuntSusie006,3.0,4.0,5.0,5.0,3.0,5.0
9,"Nov 12, 2008",NaN,NaN,5.0,Regular Guest - happy to recommend this one! F...,5.0,RoadDawg77,5.0,5.0,5.0,NaN,5.0,5.0


In [88]:
#mean of int columns
data.mean(axis=0)

Cleanliness              4.350000
Check in / front desk    4.208333
Value                    3.866667
Overall                  4.074074
Service                  4.120690
Business service         3.941176
Rooms                    3.966667
Location                 4.187500
dtype: float64